In [1]:
import pandas as pd
import csv
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from techniques1 import *
from nltk.stem.porter import *
from nltk.stem.wordnet import WordNetLemmatizer

In [2]:
def readfile(filename):
    
    arr=[]
    with open(filename,'rt',encoding='latin-1') as file:     
                samples=csv.reader(file)
                x=0
                for i in samples:
                    if x==0:
                        x=1
                        continue
                    arr.append(i)
    df=pd.DataFrame(data=arr,columns=["types","posts"])
    return df

In [3]:
def labelencode(df):
    data=df['types']
    values=array(data)
    label=LabelEncoder()
    intencode=label.fit_transform(values)
    df['typeint']=intencode
    #df['typeint'].plot(kind='hist')
    #k=np.arange(0,16)
    #x=label.inverse_transform(k)   #can access encoded actual value using x
    #print(values)
    return df


In [4]:
def createSlangframe():
    f="slang.txt"
    a=[]
    with open(f,'rt',encoding='latin-1') as file:     
                samples=csv.reader(file,delimiter='\t')
                x=0
                for i in samples:
                    if x==0:
                        x=1
                        continue
                    a.append(i)
    slangframe=pd.DataFrame(data=a,columns=['slang','fullform'])
    return slangframe


In [5]:
def preprocess(shortdata,slangframe):
    slang=list(slangframe['slang'])
    for i in range(len(shortdata)):
        #counts emojis and remove
        shortdata.loc[i,'emojicount']=countEmoticons(shortdata['posts'][i])
        shortdata.loc[i,'posts']=removeEmoticons(shortdata['posts'][i])
        #counts slang and removes
        shortdata.loc[i,'slangcount']=countSlang(shortdata['posts'][i])
        q=shortdata['posts'][i].split()
        for j in range(len(q)):
            if q[j] in slang:
                q[j]=slangframe['fullform'][slang.index(q[j])]
        shortdata.loc[i,'posts']=' '.join(q)
        #replaces contractions with full forms
        shortdata.loc[i,'posts']=replaceContraction(shortdata['posts'][i])
        #counts urls and replaces
        shortdata.loc[i,'urlcount']=countURL(shortdata['posts'][i])
        shortdata.loc[i,'posts']=replaceurl(shortdata['posts'][i]) 
        #counts #()-() and @-atuser and removes them
        shortdata.loc[i,'#_@count']=counthashAt(shortdata['posts'][i])
        shortdata.loc[i,'posts']=replaceAtUser(shortdata['posts'][i])
        shortdata.loc[i,'posts']=removeHashtagInFrontOfWord(shortdata['posts'][i])
        #puctuations clearing
        punct=clear_punctuation(shortdata['posts'][i])
        shortdata.loc[i,'posts']=punct[0]
        shortdata.loc[i,'punct_count']=punct[1]
        #replaces negations
        shortdata.loc[i,'posts']=replaceNegations(shortdata['posts'][i]) 
        #remove unicodes
        shortdata.loc[i,'posts']=removeUnicode(shortdata['posts'][i]) 
        #removes numbers
        shortdata.loc[i,'posts']=removeNumbers(shortdata['posts'][i])
        #remove stopwords
    from nltk.corpus import stopwords
    stop=stopwords.words("english")
    my_stopwords = "multiexclamation multiquestion multistop url atuser st rd nd th am pm httpurl httpsurl http" # my extra stopwords
    stop = stop + my_stopwords.split()
    shortdata['posts']=shortdata['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    ps = PorterStemmer()
    shortdata['posts'] = shortdata['posts'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split() ]))
    lmtzr = WordNetLemmatizer()
    shortdata['posts'] = shortdata['posts'].apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split() ]))
    #spell correction
    """for i in range(len(shortdata)):
        q=shortdata.loc[i,'posts'].split()
        for j in range(len(q)):
            q[j]=spellCorrection(q[j])
        shortdata.loc[i,'posts']=' '.join(q)"""
    return shortdata



In [6]:
def call_clean():
    filename="mbti_1.csv"
    df=readfile(filename)
    df=labelencode(df)
    slangframe=createSlangframe()
    clean_data=preprocess(df,slangframe)
    print(clean_data.head())
    return clean_data
data=call_clean()

  types                                              posts  typeint  \
0  INFJ  intj moment http sportscent top ten play http ...        8   
1  ENTP  Im find lack post alarmingsex bore posit often...        3   
2  INTP  good one http cours I say I know bless cursedo...       11   
3  INTJ  dear intp I enjoy convers day esoter gab natur...       10   
4  ENTJ  you firedthat anoth silli misconcept that appr...        2   

   emojicount  slangcount  urlcount  #_@count  punct_count  
0        34.0         2.0      14.0       0.0        291.0  
1        29.0         1.0       1.0       0.0        458.0  
2        19.0         0.0       4.0       0.0        339.0  
3         6.0         2.0       2.0       0.0        398.0  
4        16.0         6.0       2.0       2.0        354.0  


In [7]:
proc_data=data

In [8]:
proc_data=np.array(data['posts'])
label=np.array(data['typeint'])


In [9]:
from sklearn.model_selection import train_test_split
trainset, testset, trainlabel, testlabel = train_test_split(proc_data, label,stratify=label, test_size=0.2)
import matplotlib as plt
from matplotlib import pyplot
#plt.plot(x)
pyplot.hist(trainlabel,np.arange(0,16))


(array([ 152.,  540.,  185.,  548.,   34.,   38.,   31.,   71., 1176.,
        1465.,  873., 1043.,  133.,  217.,  434.]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
 <a list of 15 Patch objects>)

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

c:\users\nikitha\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [11]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(trainset)]
#print(tagged_data)

In [ ]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,alpha=alpha, min_alpha=0.00025,min_count=1,dm =1)
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")
print(model)
print(model.docvecs['1'])

c:\users\nikitha\appdata\local\programs\python\python36\lib\site-packages\gensim\models\doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
c:\users\nikitha\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  if sys.path[0] == '':


In [16]:
train=[]
for i in range(len(trainset)):
    train.append(model.docvecs[i])
trainset=np.array(train) 

[[-0.18914373 -0.20302776  0.00416092 ...  0.18720366  0.22891383
   0.05284937]
 [-0.3249938  -1.1973047   0.7114654  ...  0.8228047   0.62431777
   0.5789972 ]
 [-0.00142897 -0.9997056   0.4400209  ...  0.76707894  0.94090474
   1.0851221 ]
 ...
 [-0.41284165 -0.4688043   0.37761092 ...  0.4504887   0.44820946
   0.33290735]
 [-0.21763164 -0.32383454  0.24509476 ...  0.7357502   0.5944933
   0.48881269]
 [-0.46942854 -0.54872257  0.84596306 ...  0.22824056  0.9934551
   0.40817428]]


In [17]:
from gensim.models.doc2vec import Doc2Vec
test=[]
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
for i in testset:
    test_data = word_tokenize(i.lower())
    test.append(model.infer_vector(test_data))
#print("V1_infer", v1)

# to find most similar doc using tags
#similar_doc = model.docvecs.most_similar('0')
#print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
testset=np.array(test)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [15]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=6)
model.fit(trainset,trainlabel)
print(model.score(train,trainlabel))

predictionsknn = model.predict(testset)
count=0
for i in range(len(predictionsknn)):
    if predictionsknn[i]==testlabel[i]:
        count=count+1
print("accuracy=",(count*100)/len(testlabel))

0.4681556195965418
accuracy= 16.59942363112392


In [21]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(trainset,trainlabel)
print(model.score(trainset,trainlabel))
predictionsgnb=model.predict(testset)
predictionsgnb = model.predict(testset)
count=0
for i in range(len(predictionsgnb)):
    if predictionsgnb[i]==testlabel[i]:
        count=count+1
print("accuracy=",(count*100)/len(testlabel))

0.3527377521613833
accuracy= 17.23342939481268


In [22]:
from sklearn.neural_network import MLPClassifier
model=MLPClassifier()
model.fit(trainset,trainlabel)
print(model.score(trainset,trainlabel))
predictionsmlp=model.predict(testset)
predictionsmlp = model.predict(testset)
count=0
for i in range(len(predictionsmlp)):
    if predictionsmlp[i]==testlabel[i]:
        count=count+1
print("accuracy=",(count*100)/len(testlabel))

0.47521613832853027
accuracy= 20.461095100864554


c:\users\nikitha\appdata\local\programs\python\python36\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [19]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
kf = KFold(n_splits=10)
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

for train_indices, test_indices in kf.split(trainset):
    clf.fit(trainset[train_indices], trainlabel[train_indices])
    #print(clf.score(trainset[test_indices], trainlabel[test_indices]))
    prediction=clf.predict(test)
    c=0
    for i in range(len(prediction)):
        if predictionsmlp[i]==testlabel[i]:
        c=c+1
    print("accuracy=",c*100/len(testlabel))

0.30403458213256485


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.